In [ ]:
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import random
import logging

nltk.download('punkt')
nltk.download('wordnet')

logging.basicConfig(filename='chatbot.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# NLTK setup
lemmatizer = WordNetLemmatizer()

def preprocess_text_nltk(text):
    tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(token.lower()) for token in tokens]
    return lemmas

# spaCy setup
nlp = spacy.load('en_core_web_sm')

def preprocess_text_spacy(text):
    doc = nlp(text)
    lemmas = [token.lemma_.lower() for token in doc]
    return lemmas

# Define intents and responses
intents = {
    "greeting": ["hello", "hi", "hey", "good morning", "good afternoon"],
    "farewell": ["bye", "goodbye", "see you", "take care"],
    "thanks": ["thanks", "thank you", "appreciate it"],
    "weather": ["weather", "rain", "sunny", "snow"],
    "time": ["time", "clock", "current time"],
    "joke": ["joke", "funny", "tell me a joke"],
    "name": ["name", "who are you"],
    "default": ["I'm sorry, I didn't understand that."]
}

responses = {
    "greeting": ["Hello! How can I help you today?", "Hi there! How can I assist you?", "Hey! What can I do for you?"],
    "farewell": ["Goodbye! Have a great day!", "Bye! Take care!", "See you later!"],
    "thanks": ["You're welcome!", "No problem!", "Happy to help!"],
    "weather": ["I'm not sure about the weather, but I hope it's nice!", "I can't check the weather right now, sorry."],
    "time": ["I can't check the time right now, but it's always a good time to chat!", "Sorry, I can't tell you the current time."],
    "joke": ["Why don't scientists trust atoms? Because they make up everything!", "Why did the scarecrow win an award? Because he was outstanding in his field!"],
    "name": ["I'm a chatbot created to assist you!", "You can call me Chatbot."],
    "default": ["I'm sorry, I didn't understand that.", "Could you please rephrase that?", "I'm not sure I understand."]
}

context = defaultdict(str)
user_info = defaultdict(str)

def identify_intent(user_input, use_spacy=False):
    lemmas = preprocess_text_spacy(user_input) if use_spacy else preprocess_text_nltk(user_input)
    for intent, keywords in intents.items():
        if any(keyword in lemmas for keyword in keywords):
            return intent
    return "default"

def extract_entities(user_input):
    doc = nlp(user_input)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

def generate_response(user_input, use_spacy=False):
    intent = identify_intent(user_input, use_spacy)
    response = random.choice(responses.get(intent, responses["default"]))
    
    if intent == "name":
        user_name = user_info.get("name", "")
        if user_name:
            response = f"I'm a chatbot created to assist you, {user_name}!"
        else:
            response = "I'm a chatbot created to assist you!"
    
    if intent == "greeting" and "name" in user_info:
        response = f"Nice to see you again, {user_info['name']}!"
    
    return response

def chatbot(use_spacy=False):
    print("Hello! I'm your chatbot. Type 'quit' to exit.")
    logging.info("Chatbot session started.")
    
    while True:
        try:
            user_input = input("You: ")
            if user_input.lower() == 'quit':
                logging.info("Chatbot session ended.")
                break
            
            # Handle the name intent to personalize the greeting
            if "name" in user_input.lower():
                user_name = input("Please tell me your name: ")
                user_info["name"] = user_name
                response = "Got it! I will remember your name."
            else:
                response = generate_response(user_input, use_spacy)
            
            print(f"Bot: {response}")
            logging.info(f"User: {user_input}")
            logging.info(f"Bot: {response}")
        except Exception as e:
            print("Bot: Sorry, something went wrong. Please try again.")
            logging.error(f"Error: {str(e)}")

use_spacy = True
chatbot(use_spacy)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\muham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\muham\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Hello! I'm your chatbot. Type 'quit' to exit.


You:  Hey


Bot: Hi there! How can I assist you?


You:  how are you?


Bot: I'm not sure I understand.
